In [47]:
from scipy.optimize import minimize, approx_fprime
from qiskit import Aer, execute
backend = Aer.get_backend('statevector_simulator')
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.circuit import Parameter
from qiskit.quantum_info.operators import Operator
import string
import numpy as np
from scipy.linalg import expm
from functools import partial

################################
# Make a generalised 2 qubit gate
################################

def generalU(param_vals, name):
    assert len(param_vals) == 15

    sub_qr = QuantumRegister(2) # quantum - register
    sub_circ = QuantumCircuit(sub_qr, name = name)

    parameters = [Parameter(symbol) for symbol in list(map(chr, range(97, 97+15)))]

    sub_circ.rz(parameters[0], 0)
    sub_circ.ry(parameters[1], 0)
    sub_circ.rz(parameters[2], 0)

    sub_circ.rz(parameters[3], 1)
    sub_circ.ry(parameters[4], 1)
    sub_circ.rz(parameters[5], 1)

    sub_circ.cx(1,0)

    sub_circ.rz(parameters[6], 0)
    sub_circ.ry(parameters[7], 1)

    sub_circ.cx(0,1)

    sub_circ.ry(parameters[8],1 )

    sub_circ.cx(1,0)

    sub_circ.rz(parameters[9], 0)
    sub_circ.ry(parameters[10], 0)
    sub_circ.rz(parameters[11], 0)

    sub_circ.rz(parameters[12], 1)
    sub_circ.ry(parameters[13], 1)
    sub_circ.rz(parameters[14], 1)

    circ1 = sub_circ.bind_parameters({p:v for p,v in zip(parameters, param_vals)} )
    return circ1

all_params = [2*np.pi * np.random.random(15) for _ in range(8)]

U1 = generalU(all_params[0], 'U1').to_instruction()
U2 = generalU(all_params[1], 'U2').to_instruction()
U3 = generalU(all_params[2], 'U3').to_instruction()
U1_ = generalU(all_params[3], 'U1\'').to_instruction()
U2_ = generalU(all_params[4], 'U2\'').to_instruction()
U3_ = generalU(all_params[5], 'U3\'').to_instruction()

L = generalU(all_params[6], 'L').to_instruction()
R = generalU(all_params[7], 'R').to_instruction()

qr = QuantumRegister(6) # quantum - register
circ = QuantumCircuit(qr, name = 'circuit')


circ.h(qr[3])
circ.cx(3,2)

circ.barrier()

circ.append(U2, [qr[4], qr[5]])
circ.append(U1, [qr[3], qr[4]])

circ.barrier()

circ.append(R, [qr[1], qr[2]])

circ.barrier()

circ.append(U1_.inverse(), [qr[3], qr[4]])
circ.append(U2_.inverse(), [qr[4], qr[5]])

circ.barrier()

circ.cx(3,2)
circ.h(3)


print(circ.draw(fold = -1))


                   ░                ░        ░                        ░           
q331_0: ───────────░────────────────░────────░────────────────────────░───────────
                   ░                ░ ┌────┐ ░                        ░           
q331_1: ───────────░────────────────░─┤0   ├─░────────────────────────░───────────
             ┌───┐ ░                ░ │  R │ ░                        ░ ┌───┐     
q331_2: ─────┤ X ├─░────────────────░─┤1   ├─░────────────────────────░─┤ X ├─────
        ┌───┐└─┬─┘ ░        ┌─────┐ ░ └────┘ ░ ┌─────────┐            ░ └─┬─┘┌───┐
q331_3: ┤ H ├──■───░────────┤0    ├─░────────░─┤0        ├────────────░───■──┤ H ├
        └───┘      ░ ┌─────┐│  U1 │ ░        ░ │  U1'_dg │┌─────────┐ ░      └───┘
q331_4: ───────────░─┤0    ├┤1    ├─░────────░─┤1        ├┤0        ├─░───────────
                   ░ │  U2 │└─────┘ ░        ░ └─────────┘│  U2'_dg │ ░           
q331_5: ───────────░─┤1    ├────────░────────░────────────┤1        ├─░───────────
    